# Imports

In [ ]:
import pandas as pd
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Classes

In [ ]:
class ESNReservoir(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, pred_len, spectral_radius=0.9, sparsity=0.1):
        super(ESNReservoir, self).__init__()
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.pred_len = pred_len

        # Input weights
        self.Win = nn.Parameter(torch.randn(reservoir_size, input_size), requires_grad=False)

        # Reservoir weights
        self.W = nn.Parameter(torch.randn(reservoir_size, reservoir_size), requires_grad=False)

        # Output weights
        self.Wout = nn.Linear(reservoir_size, output_size)

        # Adjust spectral radius
        self.W.data *= spectral_radius / torch.max(torch.abs(torch.linalg.eigvals(self.W.data)))

        # Apply sparsity
        mask = (torch.rand(reservoir_size, reservoir_size) < sparsity).float()
        self.W.data *= mask

    def forward(self, x):
        device = x.device
        h = torch.zeros(1, self.reservoir_size).to(device)
        input_len = x.size(1)

        for t in range(input_len + self.pred_len-1):
            # take single point
            input = x[0, t, :]
            input = input.unsqueeze(0)
            # get hidden state from the point extracted and the previous hidden state
            h = torch.tanh(self.Win @ input.T + self.W @ h.T).T
            # if the time reached the input lenght we start concatenating outputs in order to pick them in the next round
            if t >= input_len-1:
                # calculate the output
                output = self.Wout(h)
                output = output.unsqueeze(1)
                x = torch.cat((x, output), dim=1)

        return x[:, -self.pred_len:, :]

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, num_layers=1, pred_len=1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.pred_len = pred_len
        self.num_layers = num_layers

        # LSTM as reservoir
        self.lstm = nn.LSTM(input_size, reservoir_size, num_layers=num_layers, batch_first=True)

        # Output weights
        self.linear1 = nn.Linear(reservoir_size, 64)
        self.linear2 = nn.Linear(64, output_size)

        self.dropout = nn.Dropout(0.2)

    # LSTM forward pass
    def forward(self, x):

        # input shape: (amount of sequences, sequences length, dimensionality of problem)
        input_len = x.size(1)
        for i in range(self.pred_len):
            # get the input and the previous outputs
            input = x[:, i:i+input_len, :]

            # the output will be just on the last hidden state
            h, _ = self.lstm(input)
            h = h[:, -1, :]
            out = F.leaky_relu(self.linear1(h))
            out = self.dropout(out)
            out = self.linear2(out)

            out = out.unsqueeze(1)
            x = torch.cat((x, out), dim=1)

        x = x[:, -self.pred_len:, :]
        return x

In [ ]:
def evaluate(num_epochs, criterion, optimizer, currentModel, train_dataloader, val_dataloader, device, scheduler=None):
    train_losses = []
    val_best_loss = np.inf
    val_best_results = {'inputs':[], 'predictions':[], 'targets':[], 'losses':[]}
    max_patience = 6
    patience = max_patience

    for epoch in range(num_epochs):
        ## begin of epoch
        print("")
        print(5*">", "New epoch", 5*"<")
        running_loss = []
        val_results = {'inputs':[], 'predictions':[], 'targets':[], 'losses':[]}
        patience-=1

        ## epoch
        # Train the model
        currentModel.train()
        for inputs, targets in train_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = currentModel(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            running_loss.append(loss.item())

        # Evaluate the model
        currentModel.eval()
        with torch.no_grad():
            for val_inputs, val_targets in val_dataloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_predictions = currentModel(val_inputs)
                val_loss = criterion(val_predictions, val_targets)
                val_results['inputs'].append(val_inputs)
                val_results['predictions'].append(val_predictions)
                val_results['targets'].append(val_targets)
                val_results['losses'].append(val_loss.cpu().item())

            val_mean_loss = np.mean(val_results["losses"])
            if val_best_loss - val_mean_loss > 1e-4: # if the best model is sensibly worst then the current
                # save best model results
                val_best_loss = val_mean_loss
                val_best_results = val_results 
                # restore patience
                patience = max_patience
                print("!!! BEST MODEL !!!")

        ## end of epoch  
        # show info
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(running_loss):.3f}, Validation loss: {val_mean_loss:.3f}')
        # scheduler
        if scheduler is not None:
            print("Learning rate: %.5f" % scheduler.get_last_lr()[0])
            scheduler.step() 
        # patience  
        if patience == 0:
            print("Ran out of patience")
            break

    return val_best_results, train_losses

In [ ]:
def loadData(dimensionality, pred_len, input_len, train_batch_size=1, val_batch_size=1, file="3BP"):
    num_files = 3

    train_sequences = torch.zeros(size=(1, input_len, dimensionality), dtype=torch.float32)
    train_targets = torch.zeros(size=(1, pred_len, dimensionality), dtype=torch.float32)
    val_sequences = torch.zeros(size=(1, input_len, dimensionality), dtype=torch.float32)
    val_targets = torch.zeros(size=(1, pred_len, dimensionality), dtype=torch.float32)


    for i in range(0, num_files):

        df = pd.read_csv(f"/kaggle/input/r3bpdataset/{file}_{i}.csv")

        if file == "3BP":
            variables = ['x', 'y',]
        elif file == "lorenz":
            variables = ['x', 'y', 'z']

        data = torch.tensor(df[variables].values)
        t = df['time'].values

        # Split the data into training and validation sets
        train_data, val_data = train_test_split(data, test_size=0.2, shuffle=False)
        train_t, val_t = train_test_split(t, test_size=0.2, shuffle=False)

        # Function to create inputs
        def create_sequences(data, pred_len, input_len, n_samples=100): # data: (points, dimension)
            inputs = torch.zeros(size=(1, input_len, data.size(1)), dtype=torch.float32) # to concat
            targets = torch.zeros(size=(1, pred_len, data.size(1)), dtype=torch.float32) # to concat

            # generate n starting points to sample n subsequences
            n_data = data.size(0)
            perm = torch.randperm(n_data - pred_len - input_len)
            starting_points = perm[:n_samples]
            for start_input in starting_points: #range(0, n_data - pred_len, input_len + pred_len):
                #if start_input + input_len + pred_len >= n_data: break # cut tail that do not fit the size of data
                

                new_input = data[start_input:start_input + input_len].unsqueeze(0)
                inputs = torch.cat((inputs, new_input), dim=0)

                new_target = data[start_input + input_len:start_input + pred_len + input_len].unsqueeze(0)
                targets = torch.cat((targets, new_target))

            # remove first entry -> zeros by initialization
            inputs = inputs[1:, :, :].float()
            targets = targets[1:, :, :].float()
            return inputs, targets

        # Create sequences for training and validation
        train_seq, train_tar = create_sequences(train_data, pred_len, input_len, n_samples=10)
        val_seq, val_tar = create_sequences(val_data, pred_len, input_len, n_samples=10)

        # Concatenate the sequences
        train_sequences = torch.cat((train_sequences, train_seq), dim=0)
        train_targets = torch.cat((train_targets, train_tar), dim=0)
        val_sequences = torch.cat((val_sequences, val_seq), dim=0)
        val_targets = torch.cat((val_targets, val_tar), dim=0)

    # remove first entry -> zeros by initialization
    train_sequences = train_sequences[1:, :, :]
    train_targets = train_targets[1:, :, :]
    val_sequences = val_sequences[1:, :, :]
    val_targets = val_targets[1:, :, :]

    # Create DataLoader for batching
    train_dataset = torch.utils.data.TensorDataset(train_sequences, train_targets)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

    val_dataset = torch.utils.data.TensorDataset(val_sequences, val_targets)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=val_batch_size, shuffle=True)

    return train_t, train_dataloader, val_t, val_dataloader

# Code

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Working on:", device)
print(30*"-")

# Define sequences length
pred_len = 100
input_len = 400

# Define the model parameters
io_size = 2
reservoir_size = 256
num_epochs = 20

### LOAD DATA
# Load the data
print("Loading data...")
train_t, train_dataloader, val_t, val_dataloader = loadData(io_size, pred_len, input_len, file="3BP")
print("Train batches:", len(train_dataloader))
print("Train input sequences:", len(train_dataloader.dataset))
print("Validation batches:", len(val_dataloader))
print("Validation input sequences:", len(val_dataloader.dataset))
print(30*"-")

In [ ]:
# init the models
model = ESNReservoir(io_size, reservoir_size, io_size, pred_len=pred_len).to(device)
modelBenchmark = LSTM(io_size, reservoir_size, io_size, num_layers=1, pred_len=pred_len).to(device)

# NMSE weighted as criterion
def NormalizedMeanSquaredError(y_pred, y_true):
    device = y_pred.get_device()
    if device == -1:
        device = 'cpu'
    pred_len = y_pred.size(1)
    batch_size = y_pred.size(0)

    squared_dist = torch.sum((y_true - y_pred)** 2, dim=2) # squared euclidean distances between predictions
    true_squared_norm = torch.sum(y_true ** 2, dim=2)
    nmse = squared_dist / true_squared_norm
    # actual (from above) shape: (batch size, prediction length)
    # WEIGHTED
    base = torch.tensor(2, dtype=torch.float32)
    weights = base.pow(-torch.arange(start=1,end=pred_len+1,step=1)).to(device)
    weights = weights/weights.sum()
    aggregated_nmse = torch.zeros(batch_size)
    for batch in range(batch_size):
        aggregated_nmse[batch] = torch.dot(nmse[batch], weights)
    # UNWEIGHTED
    # aggregated_nmse = torch.mean(torch.mean(nmse, dim=1), dim=0) 
    aggregated_nmse = torch.mean(aggregated_nmse, dim=0)
    return aggregated_nmse

# Training

In [ ]:
### RESERVOIR
# Define training setup
criterion = NormalizedMeanSquaredError
# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
print("Reservoir training...")
# start counting the time
start = time.time()
# Train the model
val_results, train_losses = (
    evaluate(num_epochs, criterion, optimizer, model, train_dataloader, val_dataloader, device, scheduler))
# stop counting the time
end = time.time()
print('Time elapsed: ', end - start, "s")
print(30*"-")

### BENCHMARK MODEL
print("Benchmark training...")
# training setup
# criterion
criterion = NormalizedMeanSquaredError
# optimizer
optimizer = optim.Adam(modelBenchmark.parameters(), lr=0.001)
# scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
# start counting the time
start = time.time()
# Train the benchmark model
val_results_benchmark, train_losses_benchmark = (
    evaluate(num_epochs, criterion, optimizer, modelBenchmark, train_dataloader, val_dataloader, device, scheduler))
# stop counting the time
end = time.time()
print('Time elapsed: ', end - start, "s")
print(30*"-")

# plot training loss
plt.plot(train_losses_benchmark)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training loss')
plt.savefig('Media/3BP_training_loss_benchmark.png')
plt.close()

# Plots

In [ ]:
### PLOTS
# Plotting the predictions
plt.figure(figsize=(15, 15))

how_many_plots = min(4, len(val_dataloader.dataset))
n_sequences = len(val_dataloader.dataset)
sequence_to_plot = torch.randint(0, n_sequences, (how_many_plots,))

batch_size = val_results['targets'][0].size(0)
batch_to_plot = torch.randint(0, batch_size, (how_many_plots,))

for plot in range(how_many_plots - how_many_plots%2):
    seq = sequence_to_plot[plot].item()
    batch = batch_to_plot[plot].item()
    # Plotting the predictions
    plt.subplot(how_many_plots // 2, 2, plot + 1)
    plt.plot(val_results['inputs'][seq][batch,:,0].cpu(), val_results['inputs'][seq][batch,:,1].cpu(), label='Input')
    plt.plot(val_results['targets'][seq][batch,:,0].cpu(), val_results['targets'][seq][batch,:,1].cpu(), label='Target')
    plt.plot(val_results['predictions'][seq][batch,:,0].cpu(), val_results['predictions'][seq][batch,:,1].cpu(), label='Predicted (Reservoir)')
    plt.plot(val_results_benchmark['predictions'][seq][batch,:,0].cpu(), val_results_benchmark['predictions'][seq][batch,:,1].cpu(), label='Predicted (Benchmark)')
    plt.xlabel('Time step')
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.savefig('Media/3BP_prediction.png')
plt.close()